<a href="https://colab.research.google.com/github/Deltaco-AB/colab-akeneo-to-ax/blob/master/Akeneo_to_AX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Akeneo to AX**

## Settings

In [ ]:
#@markdown ## Settings

download_on_complete = True #@param {type: "boolean"}

#@markdown ---
#@markdown ## Filters

filter_enabled = False #@param {type: "boolean"}
sku = ''  #@param {type: "string"}
brand_ref = ''  #@param {type: "string"}

#@markdown ---

filters = {
    "sku": sku,
    "brand_ref": brand_ref
}

### Columns

In [ ]:
output_columns = {}

## **Code**

In [ ]:
# Save converted Excel to this folder
output_dir = "converted/"

In [ ]:
!pip install xlsxwriter
import pandas as pd
from google.colab import files
from pathlib import Path

In [ ]:
# Split each filter into a whitelist
if(filter_enabled):
    for key,value in filters.items():
        if(value == ""):
            filters[key] = False
            continue
        values = value.split(",")
        filters[key] = value.split(",")

In [ ]:
# Create (stringified) AX compatible HTML elements
class AXHTML():
    @staticmethod
    def p(text):
        text = str(text)
        if(text == "nan" or len(text) < 1):
            return ""
        text = text.replace("\n\n","<br>")
        return f"<p>{text}</p>"

    @staticmethod
    def ul(items):
        ul = ""
        for li in items:
            li = str(li)
            if(li == "nan"):
                continue
            ul += f"• {li}<br>"
        return AXHTML.p(ul)

In [ ]:
# Create an Excel document
class Excel():
    def __init__(self):
        self.sheet = [[]]
        self.header = self.sheet[0]

    # Create a new column
    def append_column(self,name):
        self.header.append(name)

    # Append a complete set of cells as a row
    def insert_row(self,cells):
        if(len(cells) != len(self.header)):
            # Row item length must match header's item length
            raise ValueError("Can not insert row with different cell count")
        self.sheet.append(cells)

    # Map multidimensional list to Excel columns and rows
    def put(self,dest):
        df = pd.DataFrame(self.sheet)
        writer = pd.ExcelWriter(dest,engine="xlsxwriter")

        df.to_excel(writer,sheet_name="AkeneoToAX",index=False)
        writer.save()

# Convert Akeneo exported Excel to AX importable Excel
class AkeneoToAX(Excel):
    def __init__(self,xlsx):
        self.dataframe = pd.read_excel(xlsx)
        self.column_filter = None
        super(AkeneoToAX,self).__init__()

    # Combine all filtered fields into an AX-compatible string of HTML tags
    def format_cell(self,lang,row):
        cell = ""
        for column in self.column_filter[lang]:
            if(isinstance(column,list)):
                # Bullet point list (ul)
                items = []
                for li in column:
                    items.append(self.dataframe[li][row])
                cell += AXHTML.ul(items)
                continue
            
            # Concatinate multipart columns
            multipart = column.split("#")
            if(len(multipart) > 1):
                concat = []
                for segment in multipart:
                    data = self.dataframe[segment][row]
                    concat.append(str(data))
                cell += AXHTML.p(" ".join(concat))
                continue
            
            # Text field (paragraph)
            cell += AXHTML.p(self.dataframe[column][row])
        return cell

    # Create an array of cells
    def build_row(self,row):
        cells = []
        # Assign each cell to a column in header
        for column in self.header:
            data = ""
            if(column not in self.column_filter.keys()):
                # Treat column as array formula if not in column filter list
                cells.append(self.dataframe[column][row])
                continue
            # Run field formatting for the current cell
            cells.append(self.format_cell(column,row))
        return cells

    def exclude(self,row):
        for column,whitelist in filters.items():
            if(not whitelist or str(self.dataframe[column][row]) in whitelist):
                continue
            return True
        return False

    # Iterate over all rows in input sheet
    def walk(self):
        for i in range(0,len(self.dataframe)):
            if(filter_enabled and self.exclude(i)):
                continue
            self.insert_row(self.build_row(i))      

    # Convert columns from filter
    def columns(self,columns):
        self.column_filter = columns
        # Add each translation as column
        for lang in columns.keys():
            self.append_column(lang)

        self.walk()

In [ ]:
# Create output directory if it doesn't exists
output = Path(output_dir)
output.mkdir(parents=False,exist_ok=True)

In [ ]:
# Get all xlsx files from the current directory
xlsx_files = list(Path(".").glob("*.xlsx"))

# Convert all found xlsx files
for xlsx in xlsx_files:
    output_dest = f"converted/{xlsx}"
    convert = AkeneoToAX(xlsx)

    convert.append_column("sku") # Use SKU as key identifier
    convert.columns(output_columns)

    convert.put(output_dest)

    if(download_on_complete):
        files.download(output_dest)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>